## River figure(s): cumulative flow, weekly or bi-monthly flow

### Import modules and files

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [60]:
%cd '/home/lindsay/'

heeia_13 = pd.read_csv('heeia_13.csv', sep='\t', header=None)
heeia_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Heeia_13', 'A']
heeia_13 = pd.DataFrame.drop(heeia_13, columns=['USGS', 'ID', 'Zone', 'A'])
heeia_14 = pd.read_csv('heeia_14.csv', sep='\t', header=None)
heeia_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Heeia_14', 'A']
heeia_14 = pd.DataFrame.drop(heeia_14, columns=['USGS', 'ID', 'Zone', 'A'])
waiahole_13 = pd.read_csv('waiahole_13.csv', sep='\t', header=None) 
waiahole_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waiahole_13', 'A']
waiahole_13 = pd.DataFrame.drop(waiahole_13, columns=['USGS', 'ID', 'Zone', 'A'])
waiahole_14 = pd.read_csv('waiahole_14.csv', sep='\t', header=None)
waiahole_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waiahole_14', 'A']
waiahole_14 = pd.DataFrame.drop(waiahole_14, columns=['USGS', 'ID', 'Zone', 'A'])
waihee_13 = pd.read_csv('waihee_13.csv', sep='\t', header=None) 
waihee_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waihee_13', 'A']
waihee_13 = pd.DataFrame.drop(waihee_13, columns=['USGS', 'ID', 'Zone', 'A'])                                
waihee_14 = pd.read_csv('waihee_14.csv', sep='\t', header=None) 
waihee_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waihee_14', 'A']
waihee_14 = pd.DataFrame.drop(waihee_14, columns=['USGS', 'ID', 'Zone', 'A']) 
waikane_13 = pd.read_csv('waikane_13.csv', sep='\t', header=None)
waikane_13.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waikane_13', 'A']
waikane_13 = pd.DataFrame.drop(waikane_13, columns=['USGS', 'ID', 'Zone', 'A']) 
waikane_14 = pd.read_csv('waikane_14.csv', sep='\t', header=None) 
waikane_14.columns = ['USGS', 'ID', 'Date', 'Zone', 'Waikane_14', 'A']
waikane_14 = pd.DataFrame.drop(waikane_14, columns=['USGS', 'ID', 'Zone', 'A']) 

/home/lindsay


### Data cleaning

Given the inconsistent data lengths and sporadic missing data, I'm going to start by merging all CSVs based on the Date column.

Next, I'll separate the timestamps from the dates, then resample to mean daily values, calculate cumulative running sums, and finally, resample to mean weekly values.

In [156]:
df_13 = pd.merge(waiahole_13, waikane_13, on='Date', how='outer', sort = True)
df_13 = pd.merge(df_13, waihee_13, on='Date', how='outer', sort = True)
df_13 = pd.merge(df_13, heeia_13, on='Date', how='outer', sort = True)

df_14 = pd.merge(waiahole_14, waikane_14, on='Date', how='outer', sort = True)
df_14 = pd.merge(df_14, waihee_14, on='Date', how='outer', sort = True)
df_14 = pd.merge(df_14, heeia_14, on='Date', how='outer', sort = True)

In [157]:
# Remove timestamps
df_13['Date'] = pd.to_datetime(df_13['Date']).dt.date
df_14['Date'] = pd.to_datetime(df_14['Date']).dt.date
# Classify date col as datetime object
df_13['Date'] = pd.to_datetime(df_13['Date'])
df_14['Date'] = pd.to_datetime(df_14['Date'])
# Group by and average values by day
df_13 = df_13.set_index('Date').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')
df_14 = df_14.set_index('Date').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')

In [162]:
# Calculate cumulative sums
df_13['Waiahole_13_cs'] = df_13['Waiahole_13'].cumsum()
df_13['Waikane_13_cs'] = df_13['Waikane_13'].cumsum()
df_13['Waihee_13_cs'] = df_13['Waihee_13'].cumsum()
df_13['Heeia_13_cs'] = df_13['Heeia_13'].cumsum()

df_14['Waiahole_14_cs'] = df_14['Waiahole_14'].cumsum()
df_14['Waikane_14_cs'] = df_14['Waikane_14'].cumsum()
df_14['Waihee_14_cs'] = df_14['Waihee_14'].cumsum()
df_14['Heeia_14_cs'] = df_14['Heeia_14'].cumsum()

In [172]:
# Weekly resampling
df_13_weekly = df_13.resample('1W').mean().dropna(how='all')
df_14_weekly = df_14.resample('1W').mean().dropna(how='all')

### Plot party

For each year, I want to show the weekly discharge, and I also want to plot the running summed total for the year. 